In [56]:
%pip install neo4j python-dotenv --quiet

Note: you may need to restart the kernel to use updated packages.


# Connexion à la base Neo4J

In [57]:
import base64
import os
import socket
import time
from urllib.parse import urlparse
from neo4j import GraphDatabase
from dotenv import load_dotenv

load_dotenv()

uri = os.getenv("NEO4J_URI") or "bolt://neo4j:7687"
user = os.getenv("NEO4J_USER_DB")
password = os.getenv("NEO4J_PWD_DB")

# If host 'neo4j' is not resolvable from this environment, fallback to localhost
try:
    host = urlparse(uri).hostname
    socket.getaddrinfo(host, None)
except Exception:
    print(f"Host {host} not resolvable locally — switching to localhost")
    uri = uri.replace(host, "localhost")

if not uri or not user or not password:
    raise EnvironmentError("NEO4J_URI, NEO4J_USER_DB et NEO4J_PWD_DB doivent être définis dans .env")

print(f"CONFIGURATION : uri={uri}, user={user}, password={'*' * 8}")

# Create a driver with simple retries
def make_driver(uri, user, password, retries=5, delay=2.0):
    last_exc = None
    for attempt in range(1, retries+1):
        try:
            driver = GraphDatabase.driver(uri, auth=(user, password))
            with driver.session() as s:
                s.run("RETURN 1")
            print("Connected to Neo4j on attempt", attempt)
            return driver
        except Exception as e:
            last_exc = e
            print(f"Attempt {attempt} failed: {e}")
            time.sleep(delay)
    raise last_exc

driver = make_driver(uri, user, password)

# Example test
def get_neo4j_version(driver):
    with driver.session() as session:
        result = session.run("CALL dbms.components() YIELD versions RETURN versions[0] AS version")
        return result.single()["version"]

print("Neo4j version:", get_neo4j_version(driver))
# driver.close()

CONFIGURATION : uri=bolt://localhost:7687, user=neo4j, password=********
Connected to Neo4j on attempt 1
Neo4j version: 5.26.13


In [58]:
def get_image_data(lien):
    if not lien or lien.lower() == 'none':
        return None
    try:
        with open(lien, "rb") as img_file:
            return base64.b64encode(img_file.read()).decode("utf-8")
    except Exception:
        return None

# Hard reset

In [59]:
with driver.session() as session:
    session.run("MATCH (n) DETACH DELETE n")

# Faluchard

In [60]:
with driver.session() as session:
    definition_faluchard = "Un faluchard est un étudiant membre d'une association étudiante appelée la \"Falucharde\". Cette association est connue pour promouvoir les traditions étudiantes, notamment le port de la \"faluche\", un couvre-chef symbolique. Les faluchards participent souvent à des événements sociaux, culturels et festifs, et ils sont reconnus pour leur esprit de camaraderie et leur engagement envers la vie étudiante."
    session.run("""
        CREATE (faluchard:Faluchard {name: 'Faluchard', Definition: $definition})
    """, definition=definition_faluchard)

# Faluche

In [61]:
with driver.session() as session:
    definition_faluche = "La faluche est un couvre-chef traditionnel porté par les étudiants en France. Elle est généralement noire, en velours, et ornée de divers insignes, rubans et écussons qui représentent les affiliations académiques, les réalisations personnelles et les traditions étudiantes. La faluche est un symbole de l'identité étudiante et de la vie universitaire, souvent portée lors d'événements festifs et cérémonies."
    session.run("""
        MERGE (faluche:Faluche {name: 'Faluche', Definition: $definition})
        WITH faluche
        MATCH (faluchard:Faluchard {name: 'Faluchard'})
        MERGE (faluchard)-[:PORTER]->(faluche)
    """, definition=definition_faluche)

# Type de Faluche (Lilloise, Marseillaise, Tourangelle, ...)

In [62]:
with driver.session() as session:
    session.run("""
        CREATE (lilloise:Lilloise {name: 'Lilloise'})
        CREATE (alsacienne:Alsacienne {name: 'Alsacienne'})
        CREATE (marseillaise:Marseillaise {name: 'Marseillaise'})
        CREATE (tourangelle:Tourangelle {name: 'Tourangelle'})
        CREATE (perpignanaise:Perpignanaise {name: 'Perpignanaise'})
        CREATE (montpellieraine:Montpellieraine {name: 'Montpellieraine'})
        WITH lilloise, alsacienne, marseillaise, tourangelle, perpignanaise, montpellieraine
        MATCH (faluche:Faluche {name: 'Faluche'})
        CREATE (faluche)-[:VARIANTE]->(lilloise)
        CREATE (faluche)-[:VARIANTE]->(alsacienne)
        CREATE (faluche)-[:VARIANTE]->(marseillaise)
        CREATE (faluche)-[:VARIANTE]->(tourangelle)
        CREATE (faluche)-[:VARIANTE]->(perpignanaise)
        CREATE (faluche)-[:VARIANTE]->(montpellieraine)
    """)

# Code de la faluche

In [63]:
with driver.session() as session:
    session.run("""
        CREATE (code:Code {name: 'Code', variante: 'Lilloise'})
        WITH code
        MATCH (lilloise:Lilloise {name: 'Lilloise'})
        CREATE (lilloise)-[:A_POUR_CODE]->(code)
    """)

# Article

In [64]:
with driver.session() as session:
    # Create Articles
    session.run("""
        CREATE (a0:Article {numero: "0", titre: "Préambule", concept: "Esprit liberté, traditions étudiantes unies"})
        CREATE (a1:Article {numero: "I", titre: "Origine", concept: "Origine béret étudiant 1888"})
        CREATE (a2:Article {numero: "II", titre: "Port", concept: "Port apolitique, respect recteur"})
        CREATE (a3:Article {numero: "III", titre: "Symbolique", concept: "Pas commercial, symbole expérience étudiante"})
        CREATE (a4:Article {numero: "IV", titre: "Règles", concept: "Règles larges, uniformité fantaisie"})
        CREATE (a5:Article {numero: "V", titre: "Structure", concept: "Deux parties : ruban, velours"})
        CREATE (a6:Article {numero: "VI", titre: "Couleurs", concept: "Couleurs ruban par discipline"})
        CREATE (a7:Article {numero: "VII", titre: "Cursus", concept: "Bac, cursus, étoiles, palmes"})
        CREATE (a8:Article {numero: "VIII", titre: "Insignes", concept: "Insignes, devise, symboles personnels"})
        CREATE (a9:Article {numero: "IX", titre: "Garde santé", concept: "Faux pour décès garde santé"})
        CREATE (a10:Article {numero: "X", titre: "Rubans", concept: "Rubans ville, association, élus"})
        CREATE (a11:Article {numero: "XI", titre: "Types régionaux", concept: "Six types régionaux faluches"})
        CREATE (a12:Article {numero: "XII", titre: "Légumes intérieurs", concept: "Légumes intérieurs pour actes sexuels"})
        CREATE (a13:Article {numero: "XIII", titre: "Grand Maître", concept: "Grand Maître, garant traditions locales"})
        CREATE (a14:Article {numero: "XIV", titre: "Baptême", concept: "Baptême obligatoire par Grand Maître"})
        CREATE (a15:Article {numero: "XV", titre: "Port obligatoire", concept: "Port obligatoire cérémonies étudiantes"})
        CREATE (a16:Article {numero: "XVI", titre: "Respect faluche", concept: "Respect faluche, épreuves pour indiscrets"})
        
        // Link articles to Lilloise code
        WITH a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12, a13, a14, a15, a16
        MATCH (code:Code {name: 'Code', variante: 'Lilloise'})
        CREATE (code)-[:CONTIENT]->(a0)
        CREATE (code)-[:CONTIENT]->(a1)
        CREATE (code)-[:CONTIENT]->(a2)
        CREATE (code)-[:CONTIENT]->(a3)
        CREATE (code)-[:CONTIENT]->(a4)
        CREATE (code)-[:CONTIENT]->(a5)
        CREATE (code)-[:CONTIENT]->(a6)
        CREATE (code)-[:CONTIENT]->(a7)
        CREATE (code)-[:CONTIENT]->(a8)
        CREATE (code)-[:CONTIENT]->(a9)
        CREATE (code)-[:CONTIENT]->(a10)
        CREATE (code)-[:CONTIENT]->(a11)
        CREATE (code)-[:CONTIENT]->(a12)
        CREATE (code)-[:CONTIENT]->(a13)
        CREATE (code)-[:CONTIENT]->(a14)
        CREATE (code)-[:CONTIENT]->(a15)
        CREATE (code)-[:CONTIENT]->(a16)
    """)

# Création des concepts (Circulaire, Rubans, ...)

In [65]:
with driver.session() as session:
    session.run("""
        // RUBANS
        CREATE (ruban:Ruban {name: 'Ruban'})
        
        // RUBANS SPECIFIQUES
        CREATE (elu:Ruban {name: "Ruban d'Elu", description: "Ruban jaune pour élus étudiants"})
        CREATE (province:Ruban {name: "Ruban de Province", description: "Ruban aux couleurs de la province natale"})
        CREATE (villeEtude:Ruban {name: "Ruban de Ville d'Étude", description: "Ruban aux couleurs de la ville d'étude"})
        CREATE (villeNatale:Ruban {name: "Ruban de Ville Natale", description: "Ruban aux couleurs de la ville natale"})
        CREATE (association:Ruban {name: "Ruban d'Association", description: "Ruban aux couleurs de l'association étudiante"})
        
        WITH ruban, elu, province, villeEtude, villeNatale, association
        
        // RELATIONS RUBANS
        CREATE (elu)-[:EST_UN]->(ruban)
        CREATE (province)-[:EST_UN]->(ruban)
        CREATE (villeEtude)-[:EST_UN]->(ruban)
        CREATE (villeNatale)-[:EST_UN]->(ruban)
        CREATE (association)-[:EST_UN]->(ruban)

        // POTAGER
        CREATE (potager:Potager {name: 'Potager'})
        
        WITH ruban, potager
        
        // LIAISONS ARTICLES
        MATCH (a12:Article {numero: "XII"})
        CREATE (a12)-[:DECRIT]->(potager)
        
        WITH ruban
        
        MATCH (a10:Article {numero: "X"})
        CREATE (a10)-[:DECRIT {details: "Rubans supérieurs de gauche à droite, du frontal à l'occipital"}]->(ruban)
                
        MERGE (c:Circulaire {name: 'Circulaire'})
        WITH c
        MATCH (a6:Article {numero: "VI"})
        MERGE (a6)-[:DECRIT]->(c)
    """)

# Circulaire - Velour - Couleur - Emblème

In [66]:
with driver.session() as session:
    session.run("""
        MERGE (c:Circulaire {name: 'Circulaire'})
        MERGE (m1:Matiere {name: 'Satin'})
        MERGE (m2:Matiere {name: 'Velour'})
        MERGE (c)-[:de_matière]->(m1)
        MERGE (c)-[:de_matière]->(m2)
    """)


# Lecture du fichier Circulaire.txt (Filières;Couleur;Matière;Emblème;Image-path)
with open('TXT/Circulaire.txt', 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split(';')
        if len(parts) >= 5:
            filiere, couleur, matiere, embleme, image_path = parts[0], parts[1], parts[2], parts[3], parts[4]

            # Tentative d’ouverture et encodage Base64 de l’image
            image_data = None
            try:
                with open(image_path, "rb") as img_file:
                    image_data = base64.b64encode(img_file.read()).decode("utf-8")
            except FileNotFoundError:
                print(f"⚠️ Image non trouvée : {image_path}")
                print(f"⚠️ {filiere}, {couleur}, {matiere}, {embleme}")
            except Exception as e:
                print(f"⚠️ Erreur lors de la lecture de l’image {image_path} : {e}")

            with driver.session() as session:
                print(f"Ajout : {filiere}, {couleur}, {matiere}, {embleme}")

                # Si pas d'image encodée, ne pas créer le noeud Image ni la relation
                if image_data is None:
                    session.run(
                        """
                        MERGE (c:Circulaire {name: 'Circulaire'})
                        MERGE (m:Matiere {name: $matiere})
                        MERGE (co:Couleur {name: $couleur})
                        MERGE (d:Domaine {name: $filiere})
                        MERGE (e:Embleme {name: $embleme})

                        // Structure hiérarchique
                        MERGE (c)-[:de_matière]->(m)
                        MERGE (m)-[:de_couleur]->(co)
                        MERGE (co)-[:represente_le_domaine]->(d)

                        // Domaine → Emblème (sans Image)
                        MERGE (d)-[:a_pour_embleme]->(e)
                        """,
                        filiere=filiere,
                        couleur=couleur,
                        matiere=matiere,
                        embleme=embleme
                    )
                else:
                    session.run(
                        """
                        MERGE (c:Circulaire {name: 'Circulaire'})
                        MERGE (m:Matiere {name: $matiere})
                        MERGE (co:Couleur {name: $couleur})
                        MERGE (d:Domaine {name: $filiere})
                        MERGE (e:Embleme {name: $embleme})
                        MERGE (i:Image {path: $image_path})

                        // Si données image présentes, les ajouter
                        SET i.data = CASE WHEN $image_data IS NOT NULL THEN $image_data ELSE i.data END

                        // Structure hiérarchique
                        MERGE (c)-[:de_matière]->(m)
                        MERGE (m)-[:de_couleur]->(co)
                        MERGE (co)-[:represente_le_domaine]->(d)

                        // Domaine → Emblème → Image
                        MERGE (d)-[:a_pour_embleme]->(e)
                        MERGE (e)-[:ressemble_a]->(i)
                        """,
                        filiere=filiere,
                        couleur=couleur,
                        matiere=matiere,
                        embleme=embleme,
                        image_path=image_path,
                        image_data=image_data
                    )

Ajout : Chirurgie dentaire, Violet, Velour, Molaire
Ajout : Etudes courtes de santé, Blanc, Velour, Squelette
Ajout : Infirmier, Rose, Velour, Caducée infirmier
Ajout : Kiné, Rose, Velour, Caducée de Mercure
⚠️ Image non trouvée : None
⚠️ LAS, Marron, Velour, Emblème de la discipline majoritaire
Ajout : LAS, Marron, Velour, Emblème de la discipline majoritaire
Ajout : Médecine, Rouge, Velour, Caducée médecine
Ajout : Ostéopathie, Bleu roy, Velour, Sphénoïde
Ajout : Paramédical, Rose, Velour, Ciseaux
Ajout : Pharmacie, Vert, Velour, Caducée de pharmacie
Ajout : Préparateur en pharmacie, Vert, Velour, Mortier et Pilon
Ajout : PACES / PASS, Marron, Velour, Tête de mort sur fémurs croisés
Ajout : Sage-femme, Fuchsia, Velour, Croix d'Ânkh
Ajout : Vétérinaire, Bordeaux, Velour, Tête de cheval
Ajout : DU, Blanc, Satin, Lettres « DU »
Ajout : Administration Economique et Sociale (AES), Vert clair, Satin, Lettres « AES »
Ajout : Architecture, Bleu, Satin, Equerre & Compas
Ajout : Arts du specta

Ajout : Arts plastiques, Bleu, Satin, Palette & Pinceau
Ajout : BUT, Blanc, Satin, Lettres « BUT »
Ajout : DUT, Blanc, Satin, Lettres « DUT »
⚠️ Image non trouvée : images/Emblème_Satin/Lettres « BTS ».png
⚠️ BTS, Blanc, Satin, Lettres « BTS »
Ajout : BTS, Blanc, Satin, Lettres « BTS »
Ajout : Bachelor, Blanc, Satin, Bêta
Ajout : Classes préparatoires, Marron, Satin, Chouette bicéphale
Ajout : Communication, Suivant l'UFR de rattachement, Satin, Caducée de Mercure
Ajout : Droit, Rouge, Satin, Glaive & Balance
Ajout : Ecoles de commerce, Gestion, Communication, Journalisme, Rouge et vert, Satin, Caducée de Mercure
Ajout : Ecoles d'ingénieurs, Bleu et noir, Satin, Etoile & Foudre
⚠️ Image non trouvée : None
⚠️ Ecoles nationales, Aux couleurs du drapeau du pays de l'école, Satin, Initiales de l'école
Ajout : Ecoles nationales, Aux couleurs du drapeau du pays de l'école, Satin, Initiales de l'école
Ajout : Filières sportives, Vert foncé, Satin, Coq
Ajout : Géographie, Jaune, Satin, Globe
A

# Circulaire - Insigne Circulaire

In [67]:

# Création de Circulaire et de la catégorie Insigne
with driver.session() as session:
    session.run("""
        MERGE (c:Circulaire {name: 'Circulaire'})
    """)

# Lecture du fichier Insigne-Circulaire.txt
with open('TXT/Insigne-Circulaire.txt', 'r', encoding='utf-8') as f:
    next(f)
    for line in f:
        parts = line.strip().split(';')
        if len(parts) >= 4:
            insigne = parts[0].strip()
            signification = parts[1].strip()
            positionnement = parts[2].strip()
            lien = parts[3].strip()

            image_data = None
            try:
                with open(lien, "rb") as img_file:
                    image_data = base64.b64encode(img_file.read()).decode("utf-8")
            except FileNotFoundError:
                print(f"⚠️ Image non trouvée : {lien}")
            except Exception as e:
                print(f"⚠️ Erreur lecture image {lien} : {e}")

            with driver.session() as session:
                # Si positionnement est vide, ne pas créer ni relier le noeud Positionnement
                if positionnement == "":
                    session.run(
                        """
                        MERGE (c:Circulaire {name: 'Circulaire'})
                        MERGE (i:Insigne {name: $insigne, provenance: 'Circulaire'})
                        MERGE (s:Signification {description: $signification})
                        MERGE (img:Image {path: $lien})

                        SET img.data = CASE WHEN $image_data IS NOT NULL THEN $image_data ELSE img.data END

                        MERGE (i)-[:signifie]->(s)
                        MERGE (i)-[:a_pour_image]->(img)
                        MERGE (c)-[:contient]->(i)
                        """,
                        insigne=insigne,
                        signification=signification,
                        lien=lien,
                        image_data=image_data
                    )
                else:
                    session.run(
                        """
                        MERGE (c:Circulaire {name: 'Circulaire'})
                        MERGE (i:Insigne {name: $insigne, provenance: 'Circulaire'})
                        MERGE (s:Signification {description: $signification})
                        MERGE (p:Positionnement {description: $positionnement})
                        MERGE (img:Image {path: $lien})

                        SET img.data = CASE WHEN $image_data IS NOT NULL THEN $image_data ELSE img.data END

                        MERGE (c)-[:contient]->(i)
                        MERGE (i)-[:signifie]->(s)
                        MERGE (i)-[:est_positionne_a]->(p)
                        MERGE (i)-[:a_pour_image]->(img)
                        """,
                        insigne=insigne,
                        signification=signification,
                        positionnement=positionnement,
                        lien=lien,
                        image_data=image_data
                    )

print("✅ Import terminé avec le label correct Insigne !")

⚠️ Image non trouvée : images/Insigne/Insigne Circulaire/Drapeau de pays.png
✅ Import terminé avec le label correct Insigne !


# Insignes Autodécernés

In [68]:
# Lier Insigne_Autodecerne à Partie_Personnelle puis traitement du fichier TXT
with driver.session() as session:
    session.run("""
        MERGE (pp:Partie_Personnelle {name: 'Partie_Personnelle'})
    """)

# --- Lecture du fichier TXT ---
with open('TXT/Insigne-Autodecerne.txt', 'r', encoding='utf-8') as f:
    next(f)  # sauter l'en-tête
    for line in f:
        parts = line.strip().split(';')
        if len(parts) >= 3:
            insigne = parts[0].strip()
            signification = parts[1].strip()
            lien = parts[2].strip()

            # Lecture de l'image en base64
            image_data = None
            try:
                with open(lien, "rb") as img_file:
                    image_data = base64.b64encode(img_file.read()).decode("utf-8")
            except FileNotFoundError:
                print(f"⚠️ Image non trouvée : {lien}")
            except Exception as e:
                print(f"⚠️ Erreur lecture image {lien} : {e}")

            # Insertion dans Neo4j
            with driver.session() as session:
                session.run(
                    """
                    MATCH (pp:Partie_Personnelle {name: 'Partie_Personnelle'})
                    MERGE (i:Insigne {name: $insigne, provenance: 'Autodecerné'})
                    MERGE (s:Signification {description: $signification})
                    MERGE (img:Image {path: $lien})

                    SET img.data = CASE WHEN $image_data IS NOT NULL THEN $image_data ELSE img.data END

                    MERGE (i)-[:signifie]->(s)
                    MERGE (i)-[:a_pour_image]->(img)
                    MERGE (pp)-[:contient]->(i)
                    """,
                    insigne=insigne,
                    signification=signification,
                    lien=lien,
                    image_data=image_data
                )

print("✅ Import terminé avec succès.")


✅ Import terminé avec succès.


# Insignes Décernés

## Insigne décerné par un GM

In [69]:
with driver.session() as session:
    # 1. INITIALISATION (On crée le nœud GM unique une seule fois ici)
    session.run("""
        MERGE (pp:Partie_Personnelle {name: 'Partie_Personnelle'})
        MERGE (gm:GM {name: 'GM'}) // Création du GM central unique
    """)

    with open('TXT/Insigne-Decerne.txt', 'r', encoding='utf-8') as f:
        next(f)
        for line in f:
            parts = line.strip().split(';')
            if len(parts) >= 4:
                insigne, signification, note, lien = [p.strip() for p in parts[:4]]
                img_data = get_image_data(lien)

                # 2. TRAITEMENT PAR LIGNE
                session.run("""
                    // On récupère les parents déjà créés (MATCH est plus rapide que MERGE ici)
                    MATCH (gm:GM {name: 'GM'})
                    MATCH (pp:Partie_Personnelle {name: 'Partie_Personnelle'})
                    
                    // On crée l'insigne spécifique à cette ligne
                    MERGE (i:Insigne {name: $insigne, provenance: 'GM'})
                    MERGE (s:Signification {description: $signification})
                    MERGE (img:Image {path: $lien})
                    SET img.data = coalesce($img_data, img.data)
                    
                    MERGE (i)-[:signifie]->(s)
                    MERGE (i)-[:a_pour_image]->(img)
                    
                    // On lie l'insigne au GM unique
                    MERGE (gm)-[:decerne]->(i)
                    MERGE (pp)-[:contient]->(i)
                    
                    FOREACH (_ IN CASE WHEN $note <> '' AND toLower($note) <> 'none' THEN [1] ELSE [] END |
                        MERGE (a:A_noter {a_noter: $note})
                        MERGE (i)-[:a_pour_note]->(a)
                    )
                """, insigne=insigne, signification=signification, note=note, lien=lien, img_data=img_data)

## Insigne décerné par un partenaire

In [70]:
with driver.session() as session:
    session.run("""
        MERGE (pp:Partie_Personnelle {name: 'Partie_Personnelle'})
    """)

    with open('TXT/Insigne-Partenaire.txt', 'r', encoding='utf-8') as f:
        next(f)
        for line in f:
            parts = line.strip().split(';')
            if len(parts) >= 3:
                insigne, signification, lien = [p.strip() for p in parts[:3]]
                img_data = get_image_data(lien)

                # MERGE avec contexte 'Partenaire'
                session.run("""
                    MATCH (pp:Partie_Personnelle {name: 'Partie_Personnelle'})
                    MERGE (i:Insigne {name: $insigne, provenance: 'Partenaire'})
                    MERGE (s:Signification {description: $signification})
                    MERGE (img:Image {path: $lien})
                    SET img.data = coalesce($img_data, img.data)
                    
                    MERGE (pp)-[:contient]->(i)
                    MERGE (i)-[:signifie]->(s)
                    MERGE (i)-[:a_pour_image]->(img)
                """, insigne=insigne, signification=signification, lien=lien, img_data=img_data)

# Insignes Regionaux

In [71]:
with driver.session() as session:
    with open('TXT/Insigne-Regionaux.txt', 'r', encoding='utf-8') as f:
        next(f)  # sauter l'en-tête
        for line in f:
            parts = line.strip().split(';')
            if len(parts) >= 3:
                insigne = parts[0].strip()
                signification = parts[1].strip()
                lien = parts[2].strip()

                # Lecture de l'image en base64
                image_data = None
                try:
                    with open(lien, "rb") as img_file:
                        image_data = base64.b64encode(img_file.read()).decode("utf-8")
                except FileNotFoundError:
                    print(f"⚠️ Image non trouvée : {lien}")
                except Exception as e:
                    print(f"⚠️ Erreur lecture image {lien} : {e}")

                # Insertion dans Neo4j — lier l'insigne à la catégorie Insigne_Regionaux qui est attachée au Ruban de province
                with driver.session() as session:
                    session.run(
                        """
                        MATCH (rp:Ruban {name: "Ruban de Province", description: "Ruban aux couleurs de la province natale"})
                        MERGE (i:Insigne {name: $insigne, provenance: 'Régional'})
                        MERGE (s:Signification {description: $signification})
                        MERGE (img:Image {path: $lien})
                        SET img.data = CASE WHEN $image_data IS NOT NULL THEN $image_data ELSE img.data END
                        MERGE (rp)-[:contient]->(i)
                        MERGE (i)-[:signifie]->(s)
                        MERGE (i)-[:a_pour_image]->(img)
                        """,
                        insigne=insigne,
                        signification=signification,
                        lien=lien,
                        image_data=image_data
                    )

    print("✅ Import Insignes Régionaux terminé avec succès.")

✅ Import Insignes Régionaux terminé avec succès.


# Insignes Ruban d'élu

In [72]:
# Lier Partie_Personnelle -> Ruban d'elu -> Insigne_Elu puis traitement du fichier TXT
with driver.session() as session:
    with open('TXT/Insigne-Elu.txt', 'r', encoding='utf-8') as f:
        next(f)  # sauter l'en-tête
        for line in f:
            parts = line.strip().split(';')
            if len(parts) >= 3:
                insigne = parts[0].strip()
                signification = parts[1].strip()
                lien = parts[2].strip()

                # Lecture de l'image en base64 (si chemin présent et non 'None')
                image_data = None
                try:
                    if lien and lien.lower() != 'none':
                        with open(lien, "rb") as img_file:
                            image_data = base64.b64encode(img_file.read()).decode("utf-8")
                except FileNotFoundError:
                    print(f"⚠️ Image non trouvée : {lien}")
                except Exception as e:
                    print(f"⚠️ Erreur lecture image {lien} : {e}")

                with driver.session() as session:
                    session.run(
                        """
                        MATCH (elu:Ruban {name: "Ruban d'Elu", description: "Ruban jaune pour élus étudiants"})
                        MERGE (i:Insigne {name: $insigne, provenance: "Ruban d'élu"})
                        MERGE (s:Signification {description: $signification})
                        MERGE (img:Image {path: $lien})
                        SET img.data = CASE WHEN $image_data IS NOT NULL THEN $image_data ELSE img.data END
                        MERGE (i)-[:signifie]->(s)
                        MERGE (i)-[:a_pour_image]->(img)
                        MERGE (elu)-[:contient]->(i)
                        """,
                        insigne=insigne,
                        signification=signification,
                        lien=lien,
                        image_data=image_data
                    )

    print("✅ Import Insignes Elu terminé avec succès.")

✅ Import Insignes Elu terminé avec succès.


# Insignes Potagers

In [73]:
# Lier Insigne_Potager à Potager puis traitement du fichier TXT
with driver.session() as session:
    session.run("""
        MERGE (pot:Potager {name: 'Potager'})
    """)

with open('TXT/Insigne-Potager.txt', 'r', encoding='utf-8') as f:
    next(f)  # sauter l'en-tête
    for line in f:
        parts = line.strip().split(';')
        if len(parts) >= 3:
            insigne = parts[0].strip()
            signification = parts[1].strip()
            lien = parts[2].strip()

            # Lecture de l'image en base64
            image_data = None
            try:
                with open(lien, "rb") as img_file:
                    image_data = base64.b64encode(img_file.read()).decode("utf-8")
            except FileNotFoundError:
                print(f"⚠️ Image non trouvée : {lien}")
            except Exception as e:
                print(f"⚠️ Erreur lecture image {lien} : {e}")

            # Insertion dans Neo4j
            with driver.session() as session:
                session.run(
                    """
                    MATCH (pot:Potager {name: 'Potager'})
                    MERGE (i:Insigne {name: $insigne,  provenance: 'Potager'})
                    MERGE (s:Signification {description: $signification})
                    MERGE (img:Image {path: $lien})
                    SET img.data = CASE WHEN $image_data IS NOT NULL THEN $image_data ELSE img.data END
                    MERGE (i)-[:signifie]->(s)
                    MERGE (i)-[:a_pour_image]->(img)
                    MERGE (pot)-[:contient]->(i)
                    """,
                    insigne=insigne,
                    signification=signification,
                    lien=lien,
                    image_data=image_data
                )

print("✅ Import Insignes Potagers terminé avec succès.")

✅ Import Insignes Potagers terminé avec succès.
